In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

In [7]:
df = pd.read_csv("historic.csv")
df.head()

item_no    category      main_promotion  color  stars success_indicator
0   739157       Tunic             Catalog  Green    3.1              flop
1   591846      Hoodie  Category_Highlight    Red    1.5              flop
2   337574  Sweatshirt             Catalog    Red    4.4               top
3   401933  Polo-Shirt  Category_Highlight   Blue    3.1              flop
4   812151      Hoodie  Category_Highlight  Green    4.1               top

In [10]:
# Encode categorical columns
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])
df['main_promotion_encoded'] = label_encoder.fit_transform(df['main_promotion'])
df['color_encoded'] = label_encoder.fit_transform(df['color'])
df['success_indicator_encoded'] = label_encoder.fit_transform(df['success_indicator'])
df.head()

item_no    category      main_promotion  color  stars success_indicator  \
0   739157       Tunic             Catalog  Green    3.1              flop   
1   591846      Hoodie  Category_Highlight    Red    1.5              flop   
2   337574  Sweatshirt             Catalog    Red    4.4               top   
3   401933  Polo-Shirt  Category_Highlight   Blue    3.1              flop   
4   812151      Hoodie  Category_Highlight  Green    4.1               top   

   category_encoded  main_promotion_encoded  color_encoded  \
0                 5                       0              3   
1                 1                       1              7   
2                 3                       0              7   
3                 2                       1              1   
4                 1                       1              3   

   success_indicator_encoded  
0                          0  
1                          0  
2                          1  
3                          0  
4                          1

In [11]:
# Define features and target
features = df[['category_encoded', 'main_promotion_encoded', 'color_encoded', 'stars']]
target = df['success_indicator_encoded']

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Feature Scaling for non-ANN models
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# 1. Decision Tree
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
dt_predictions = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print(f'Decision Tree Accuracy: {dt_accuracy}')

Decision Tree Accuracy: 0.798125


In [14]:
# 2. Random Forest
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f'Random Forest Accuracy: {rf_accuracy}')

Random Forest Accuracy: 0.825625


In [15]:
# 3. Artificial Neural Network (ANN)
# Convert target to categorical for ANN
target_categorical = to_categorical(target)

# Build the ANN model
ann_model = Sequential([
    Dense(64, activation='relu', input_shape=(features.shape[1],)),
    Dense(32, activation='relu'),
    Dense(2, activation='softmax')  # Assuming binary classification, adjust for multiclass
])

In [17]:
# Compile the model
ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
ann_model.fit(X_train_scaled, target_categorical, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the ANN model
ann_predictions_probs = ann_model.predict(X_test_scaled)
ann_predictions = np.argmax(ann_predictions_probs, axis=1)
ann_accuracy = accuracy_score(y_test, ann_predictions)
print(f'ANN Accuracy: {ann_accuracy}')

Epoch 1/10
160/160 [==============================] - 2s 5ms/step - loss: 0.6430 - accuracy: 0.6500 - val_loss: 0.6508 - val_accuracy: 0.6500
Epoch 2/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6411 - accuracy: 0.6504 - val_loss: 0.6510 - val_accuracy: 0.6516
Epoch 3/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6415 - accuracy: 0.6502 - val_loss: 0.6521 - val_accuracy: 0.6508
Epoch 4/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6411 - accuracy: 0.6504 - val_loss: 0.6538 - val_accuracy: 0.6516
Epoch 5/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6412 - accuracy: 0.6510 - val_loss: 0.6536 - val_accuracy: 0.6469
Epoch 6/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6402 - accuracy: 0.6512 - val_loss: 0.6533 - val_accuracy: 0.6516
Epoch 7/10
160/160 [==============================] - 0s 2ms/step - loss: 0.6398 - accuracy: 0.6494 - val_loss: 0.6528 - val_accuracy: 0.6484
Epoch 

We have trained three model on different algorithm on the same dataset. The algorithm used are decision tree, random forest and ANN. Random forest algorithm gave the highest accuracy out of all three algorithm.